In [9]:
import tasks
from utils import parse_config

configuration = parse_config("config")

raw_data = configuration["raw_data"]
TABLES = configuration["TABLES"]
FACT_TABLES = configuration["FACT_TABLES"]
SCD_types = configuration["SCD"]

db_master = configuration["db_master"]

db_rel = configuration["db_rel"]
db_dim = configuration["db_dim"]

schema_rel = configuration["schema_rel"]
schema_dim = configuration["schema_dim"]

In [10]:
conn_master = tasks.connect_to(db_master)

tasks.drop_database_if_exists(conn_master)
tasks.create_database(conn_master)

conn_master.close()

Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=master;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    

Dropped databases
Created databases


In [11]:
conn_ER = tasks.connect_to(db_rel)

tasks.create_database(conn_ER)

tasks.drop_constraints_if_exist(conn_ER, db_rel, schema_rel)

for table in TABLES:
    tasks.drop_table_if_exists(conn_ER, table, db_rel, schema_rel)
    tasks.create_table(conn_ER, table, db_rel, schema_rel)
    tasks.insert_into_table(conn_ER, raw_data, table, db_rel, schema_rel)

tasks.set_constraints(conn_ER, db_rel, schema_rel)

conn_ER.close()

Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_RELATIONAL_DB;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    

Created databases
The dbo.Categories table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Categories table from the database Orders_RELATIONAL_DB has been created
Inserting into Categories...
8 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Categories table

The dbo.Customers table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Customers table from the database Orders_RELATIONAL_DB has been created
Inserting into Customers...
91 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Customers table

The dbo.Employees table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Employees table from the database Orders_RELATIONAL_DB has been created
Inserting into Employees...
9 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Emp

In [12]:
conn_DW = tasks.connect_to(db_dim)

for table, SCD_type in SCD_types.items():
    
    tasks.drop_dim_table_if_exists(conn_DW, table, SCD_type, db_dim, schema_dim)
    
    tasks.create_dim_table(conn_DW, table, SCD_type, db_dim, schema_dim)
    
    tasks.update_dim_table(conn_DW, table, SCD_type, db_dim, db_rel, schema_dim, schema_rel)

for fact_table in FACT_TABLES:
    tasks.drop_fact_table_if_exists(conn_DW, fact_table, db_dim, schema_dim)
    tasks.create_fact_table(conn_DW, fact_table, db_dim, schema_dim)
    tasks.update_fact_table(conn_DW, fact_table, db_dim, db_rel, schema_dim, schema_rel)

conn_DW.close()

Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_DIMENSIONAL_DW;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    

The dbo.dim_Categories_SCD3 table from the database Orders_DIMENSIONAL_DW has been dropped
The dbo.dim_Categories_SCD3 table from the database Orders_DIMENSIONAL_DW has been created

Executing Categories_SCD3...
Updated dim table Categories_SCD3

The dbo.dim_Customers_SCD3 table from the database Orders_DIMENSIONAL_DW has been dropped
The dbo.dim_Customers_SCD3 table from the database Orders_DIMENSIONAL_DW has been created

Executing Customers_SCD3...
Updated dim table Customers_SCD3

The dbo.dim_Employees_SCD2 table from the database Orders_DIMENSIONAL_DW has been dropped
The dbo.dim_Employees_SCD2 table from the database Orders_DIMENSIONAL_DW has been created

Executing Employees_SCD2...
Updated dim table Employees_SCD2

The dbo.dim_Products_SCD4 table from the database Orders_DIMENSIONAL_D

In [13]:
conn_ER = tasks.connect_to(db_rel)
exec_cursor = conn_ER.execute("select * from dbo.Territories")
print(exec_cursor.fetchall())
conn_ER.close()

Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_RELATIONAL_DB;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    

[(1581, 'Westboro', 1), (1730, 'Bedford', 1), (1833, 'Georgetow', 1), (2116, 'Boston', 1), (2139, 'Cambridge', 1), (2184, 'Braintree', 1), (2903, 'Providence', 1), (3049, 'Hollis', 3), (3801, 'Portsmouth', 3), (6897, 'Wilton', 1), (7960, 'Morristown', 1), (8837, 'Edison', 1), (10019, 'New York', 1), (10038, 'New York', 1), (11747, 'Mellvile', 1), (14450, 'Fairport', 1), (19428, 'Philadelphia', 3), (19713, 'Neward', 1), (20852, 'Rockville', 1), (27403, 'Greensboro', 1), (27511, 'Cary', 1), (29202, 'Columbia', 4), (30346, 'Atlanta', 4), (31406, 'Savannah', 4), (32859, 'Orlando', 4), (33607, 'Tampa', 4), (40222, 'Louisville', 1), (44122, 'Beachwood', 3), (45839, 'Findlay', 3), (48075, 'Southfield', 3), (48084, 'Troy', 3), (48304, 'Bloomfield Hills', 3), (53404, 'Racine', 3), (55113, 'Roseville', 

In [14]:
import csv
import os

def write_csv_to_file(db, schema, table):

    tablename = f"{db}.{schema}.{table}"
    filename = f'blobs/{tablename}.csv'

    conn = tasks.connect_to(db)
    conn_cursor = conn.cursor()
    rows = conn_cursor.execute(f"select * from {tablename}")
    
    os.makedirs(os.path.dirname(filename), exist_ok = True)
    
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        row = [x[0] for x in conn_cursor.description]
        writer.writerow(row)  # column headers
        for row in rows:
            writer.writerow(row)

    conn.close()

In [15]:
for table, SCD_type in SCD_types.items():
    table_name = f"dim_{table}_{SCD_type}"
    print(table_name)
    write_csv_to_file(db_dim, schema_dim, table_name)

dim_Categories_SCD3
Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_DIMENSIONAL_DW;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    

dim_Customers_SCD3
Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_DIMENSIONAL_DW;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    

dim_Employees_SCD2
Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_DIMENSIONAL_DW;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    

dim_Products_SCD4
Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_DIMENSIONAL_DW;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    

dim_Region_SCD1_with_delete
Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_DIME

In [16]:
for table in FACT_TABLES:
    table_name = f"fact_{table}"
    print(table_name)
    write_csv_to_file(db_dim, schema_dim, table_name)

fact_Orders
Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_DIMENSIONAL_DW;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    



In [17]:
write_csv_to_file(db_dim, schema_dim, "dim_Products_SCD4_History")

Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_DIMENSIONAL_DW;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    



In [18]:
t = "blobs/Orders_DIMENSIONAL_DW.dbo.dim_Categories_SCD3"
import pandas as pd
pd.read_csv(t)

,CategoryID_SK,CategoryID_NK,CategoryName,Description,Description_Prev1,Description_Prev1_ValidTo
0,1,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",NaN,NaN
1,2,2,Condiments,"Sweet and savory sauces, relishes, spreads, an...",NaN,NaN
2,3,3,Confections,"Desserts, candies, and sweet breads",NaN,NaN
3,4,4,Dairy Products,Cheeses,NaN,NaN
4,5,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",NaN,NaN
5,6,6,Meat/Poultry,Prepared meats,NaN,NaN
6,7,7,Produce,Dried fruit and bean curd,NaN,NaN
7,8,8,Seafood,Seaweed and fish,NaN,NaN
